In [1]:
%matplotlib widget
import obspy
from obspy.signal.trigger import classic_sta_lta, recursive_sta_lta, aic_simple
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import coincidence_trigger
from obspy.signal.cross_correlation import correlation_detector
from obspy.signal import PPSD
from obspy import read_inventory
from obspy.realtime.signal import kurtosis
from obspy.signal import filter



from hydrophone_data_processing import load, useful_variables, plotting, signal_processing
import obspy.signal.trigger as trigger
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plot_working
import event_processing
import loading_stream_function

import matplotlib.ticker as ticker
from datetime import datetime
from datetime import timedelta

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
chan = 'GHZ'
#statlist = [ 'A01' ,'A03' , 'A05' , 'A07' , 'A09' , 'A11' , 'A13' , 'A15' , 'A17' , 'A19' ]
statlist = [ 'A01' ,'A02' , 'A03' , 'A04' , 'A05' , 'A06' , 'A07' , 'A08' , 'A09', 'A10' , 'A11', 'A13', 'A14', 'A19']#, 'A16', 'A17', 'A18', 'A19' ]
#statlist = [ 'B01' ,'B02' , 'B03' , 'B04' , 'B06' , 'B07' , 'B08' , 'B09','B10' , 'B11', 'B12', 'B13' , 'B19' , 'B20']#,'B14','B15','B16','B17','B18','B19', 'B20' ]

day = 15
stream = obspy.read("/media/sbrent/Oman3/PASSCAL/Main_deployment/DAYS/" +statlist[0]+ "/*.." + chan +".2020.0" +str(day))


In [ ]:
for s in statlist[1:]:
    
    stream += obspy.read("/media/sbrent/Oman3/PASSCAL/Main_deployment/DAYS/" + s + "/*.." + chan + ".2020.0" + str(day))


In [ ]:
hr = 15
start_time = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+hr*3600
end_time = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+(hr+1)*3600
data15 = stream.slice(starttime=start_time,endtime=end_time) #look into trim()
#print(data)
print(start_time, end_time)

In [ ]:
data15_cp = data15.copy()
data15_cp2 = data15.copy()
data15_cp3 = data15.copy()
data15_cp4 = data15.copy()
data15_cp5 = data15.copy()
data15_cp6 = data15.copy()

In [ ]:

stream_filt = data15.filter('highpass', corners=4, freq=80, zerophase=False)
stream_2 = stream_filt.copy()
events = coincidence_trigger("classicstalta", 5, 0.8, stream_2, 3, sta=0.02, lta=0.2)
events = pd.DataFrame(events)

#g_wind1 = event_processing.get_event_window_raw(data15_cp,TIME)

In [ ]:
event_idx = 509
TIME = events.time[event_idx]
print(TIME)
g_wind1 = event_processing.get_event_window_raw(data15_cp,TIME)
g_wind1 = g_wind1.detrend('demean')

In [ ]:
event_idx = 509

In [ ]:
event_idx1 = 147
event_idx2 = 992
event_idx3 = 1095
TIME_EV2 = events.time[event_idx1]
print(TIME_EV2)
TIME_EV3 = events.time[event_idx2]
print(TIME_EV3)
TIME_EV4 = events.time[event_idx3]
print(TIME_EV4)

g_wind2 = event_processing.get_event_window_raw(data15_cp3,TIME_EV2)
g_wind2 = g_wind2.detrend('demean')
g_wind3 = event_processing.get_event_window_raw(data15_cp4,TIME_EV3)
g_wind3 = g_wind3.detrend('demean')
g_wind4 = event_processing.get_event_window_raw(data15_cp5,TIME_EV4)
g_wind4 = g_wind4.detrend('demean')

In [ ]:
g_wind3

In [ ]:
events.time[event_idx3]

In [ ]:
events.coincidence_sum[event_idx3]

In [ ]:
events.trace_ids[event_idx3]

In [ ]:
#def events_coincidence (coin_sum, time):
#    df[coin_sum]= np.where((df[coin_sum])>9.0, df[time])
#    return df

In [ ]:
#df = events_coincidence('coincidence_sum', 'time')

In [ ]:
class PrecisionDateFormatter(ticker.Formatter):
    """
    Extend the `matplotlib.ticker.Formatter` class to allow for millisecond
    precision when formatting a tick (in days since the epoch) with a
    `~datetime.datetime.strftime` format string.

    """

    def __init__(self, fmt, precision=3, tz=None):
        """
        Parameters
        ----------
        fmt : str
            `~datetime.datetime.strftime` format string.
        """
        from matplotlib.dates import num2date
        if tz is None:
            from matplotlib.dates import _get_rc_timezone
            tz = _get_rc_timezone()
        self.num2date = num2date
        self.fmt = fmt
        self.tz = tz
        self.precision = precision

    def __call__(self, x, pos=0):
        if x == 0:
            raise ValueError("DateFormatter found a value of x=0, which is "
                             "an illegal date; this usually occurs because "
                             "you have not informed the axis that it is "
                             "plotting dates, e.g., with ax.xaxis_date()")

        dt = self.num2date(x, self.tz)
        ms = dt.strftime("%f")[:self.precision]

        return dt.strftime(self.fmt).format(ms=ms)

    def set_tzinfo(self, tz):
        self.tz = tz

In [ ]:
fig, ax = plt.subplots(1, figsize=(17, 10), sharex=True, sharey=False)
ax.plot(data15_cp2[0].times('matplotlib'), data15_cp2[0].data)
ax.xaxis.set_major_formatter(PrecisionDateFormatter("%H:%M:%S.{ms}"))

In [ ]:
TIME1 = obspy.UTCDateTime('2020-1-' + str(day) + 'T15:20:14')
noise1 = event_processing.get_event_window_raw(data15_cp,TIME1)

TIME2 = obspy.UTCDateTime('2020-1-' + str(day) + 'T15:20:15')
noise2 = event_processing.get_event_window_raw(data15_cp,TIME2)

hr = 16
start_time1 = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+hr*3600
end_time1 = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+(hr+1)*3600
data16 = stream.slice(starttime=start_time1,endtime=end_time1) #look into trim()

TIME3 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:16:40')
noise3 = event_processing.get_event_window_raw(data16,TIME3)

TIME4 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:16:45')
noise4 = event_processing.get_event_window_raw(data16,TIME4)

TIME5 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:16:49')
noise5 = event_processing.get_event_window_raw(data16,TIME5)

TIME6 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:16:53')
noise6 = event_processing.get_event_window_raw(data16,TIME6)

TIME7 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:50:26')
noise7 = event_processing.get_event_window_raw(data16,TIME7)


TIME8 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:50:31')
noise8 = event_processing.get_event_window_raw(data16,TIME8)

TIME9 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:50:35')
noise9 = event_processing.get_event_window_raw(data16,TIME9)


TIME10 = obspy.UTCDateTime('2020-1-' + str(day) + 'T16:02:10')
noise10 = event_processing.get_event_window_raw(data16,TIME10)

In [ ]:
from spectrum import *
def multi_taper(N, NW, k, dt, data):
    "This function depends on spectrum so it needs to be imported"
    [tapers, eigen] = dpss(N, NW, k)
    Sk_complex, weights, eigenvalues=pmtm(data, e=eigen, v=tapers, NFFT=N, show=False)

    Sk = abs(Sk_complex)**2
    Sk = np.mean(Sk * np.transpose(weights), axis=0) * dt
    #print(Sk[0])
    return Sk

In [ ]:
N = len(g_wind1[0].data)
NW = 3/2
k = 8
data = g_wind1[0].data
dt=g_wind1[0].stats.delta

data_n1 = noise1[0].data
data_n2 = noise2[0].data
data_n3 = noise3[0].data
data_n4 = noise4[0].data
data_n5 = noise5[0].data
data_n6 = noise6[0].data
data_n7 = noise7[0].data
data_n8 = noise8[0].data
data_n9 = noise9[0].data
data_n10 = noise10[0].data

In [ ]:
mtp_n1 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n1)
mtp_n2 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n2)
mtp_n3 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n3)
mtp_n4 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n4)
mtp_n5 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n5)
mtp_n6 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n6)
mtp_n7 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n7)
mtp_n8 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n8)
mtp_n9 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n9)
mtp_n10 = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_n10)

average_noise_mtm = (mtp_n1 + mtp_n2 + mtp_n3 + mtp_n4 + mtp_n5 + mtp_n6 + mtp_n7 + mtp_n8 + mtp_n9 + mtp_n10)/10

In [ ]:
data_cp1 = g_wind1.copy()
data_cp2 = g_wind1.copy()
data_cp3 = g_wind1.copy()
data_cp4 = g_wind1.copy()
data_hp = data_cp1.filter(type='highpass', corners=4, zerophase=True, freq=20)
data_hp_cp = data_hp.copy()
data_lp = data_hp_cp.filter(type='lowpass', corners=4, zerophase=True, freq=200)

data_hp350 = data_cp1.filter(type='highpass', corners=4, zerophase=True, freq=350)
#data_hp350 = data_hp350[0].data

data_lp250 = data_cp4.filter(type='lowpass', corners=4, zerophase=True, freq=250)
#data_lp250 = data_lp250[0].data


data_bp = data_cp2.filter(type="bandpass", freqmin=100, freqmax=250)  
#data_bp = data_bp[0].data

In [ ]:
data_ev2_cp1 = g_wind2.copy()
data_ev2_cp2 = g_wind2.copy()
data_ev2_cp3 = g_wind2.copy()
data_ev2_hp350 = data_ev2_cp2.filter(type='highpass', corners=4, zerophase=True, freq=350)
data_ev2_bp = data_ev2_cp3.filter(type="bandpass", freqmin=100, freqmax=250)  

In [ ]:
data_ev3_cp1 = g_wind3.copy()
data_ev3_cp2 = g_wind3.copy()
data_ev3_cp3 = g_wind3.copy()
data_ev3_hp350 = data_ev3_cp2.filter(type='highpass', corners=4, zerophase=True, freq=350)
data_ev3_bp = data_ev3_cp3.filter(type="bandpass", freqmin=100, freqmax=250)  

In [ ]:
data_ev4_cp1 = g_wind4.copy()
data_ev4_cp2 = g_wind4.copy()
data_ev4_cp3 = g_wind4.copy()
data_ev4_hp350 = data_ev4_cp2.filter(type='highpass', corners=4, zerophase=True, freq=350)
data_ev4_bp = data_ev4_cp3.filter(type="bandpass", freqmin=100, freqmax=250)  

In [ ]:
data_lp_hp = data_hp[0].data
data_lp_hp_mtm = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_lp_hp)

data_bp = data_bp[0].data
data_bp_mtm = multi_taper(N=len(g_wind1[0].data), NW=1.0, k=8, dt = g_wind1[0].stats.delta, data = data_bp)

In [ ]:
from spectrum import *

x = np.linspace(0.0, N*dt, N)
y = data
plt.figure()
plt.plot(x,y, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(x, data_hp350, 'blue', label = "hp350")
plt.plot(x, data_bp, 'purple', label = "bp100-250")

plt.legend(loc="upper right")

# classical FFT
xf = np.linspace(0.0, 1.0/(2.0*dt), N//2)
y_event = np.fft.fft(y)

#y_n1 = np.fft.fft(data_n1)
#y_n2 = np.fft.fft(data_n2)
#y_n3 = np.fft.fft(data_n3)
#y_n4 = np.fft.fft(data_n4)
#y_n5 = np.fft.fft(data_n5)
#y_n6 = np.fft.fft(data_n6)
#y_n7 = np.fft.fft(data_n7)
#y_n8 = np.fft.fft(data_n8)
#y_n9 = np.fft.fft(data_n9)
#y_n10 = np.fft.fft(data_n10)


# The multitapered method
NW=1.0
k=8
[tapers, eigen] = dpss(N, NW, k)
Sk_complex, weights, eigenvalues=pmtm(y, e=eigen, v=tapers, NFFT=N, show=False)

Sk = abs(Sk_complex)**2
Sk = np.mean(Sk * np.transpose(weights), axis=0) * dt
print(Sk[0])
# ploting both results
#plt.figure()
#plt.plot(xf,abs(y_event[0:N//2])**2 / N * dt, 'limegreen')
#plt.plot(xf,Sk[0:N//2], 'dodgerblue')

# ploting both results in log scale
plt.figure()
plt.semilogy(xf, abs(y_event[0:N // 2]) ** 2 / N * dt, 'limegreen')
plt.semilogy(xf, Sk[0:N // 2], 'dodgerblue', label = "main event")

plt.semilogy(xf, mtp_n1[0:N // 2], 'blue', label = "noise1")
plt.semilogy(xf, mtp_n2[0:N // 2], 'orange', label = "noise2")
plt.semilogy(xf, mtp_n3[0:N // 2], 'green', label = "noise3")
plt.semilogy(xf, mtp_n4[0:N // 2], 'red', label = "noise4")
plt.semilogy(xf, mtp_n5[0:N // 2], 'purple', label = "noise5")
plt.semilogy(xf, mtp_n6[0:N // 2], 'brown', label = "noise6")
plt.semilogy(xf, mtp_n7[0:N // 2], 'pink', label = "noise7")
plt.semilogy(xf, mtp_n8[0:N // 2], 'gray', label = "noise8")
plt.semilogy(xf, mtp_n9[0:N // 2], 'cyan', label = "noise9")
plt.semilogy(xf, mtp_n10[0:N // 2], 'turquoise', label = "noise10")
plt.legend(loc="upper right")




plt.figure()
#plt.semilogy(xf, abs(y_event[0:N // 2]) ** 2 / N * dt, 'limegreen')
plt.semilogy(xf, Sk[0:N // 2], 'dodgerblue', label = "main event")data_bp
plt.semilogy(xf, data_lp_hp_mtm[0:N // 2], 'royalblue', label = "main event_hp_lp")
plt.semilogy(xf, data_bp_mtm[0:N // 2], 'green', label = "main event_bp")
plt.semilogy(xf, average_noise_mtm[0:N // 2], 'turquoise', label = "averaged_noise")
plt.legend(loc="upper right")

In [ ]:
df_time = events.time
df_duration = events.duration
ct_stations = events.stations
ct_sum = events.coincidence_sum

In [ ]:
len(events.trace_ids.iloc[event_idx1])

In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

fig, ax = plt.subplots(14, figsize=(30, 75), sharex=True, sharey=False)

for id in events.trace_ids.iloc[event_idx1]:
    
    i = int(axes_id[id])
    print(i)
    ax[i].plot(g_wind2[i].times('matplotlib'), g_wind2[i].data, color='limegreen', label="event_unfiltered")
    ax[i].plot(data_ev2_hp350[i].times('matplotlib'), data_ev2_hp350[i].data, 'blue', label = "hp350")
    ax[i].plot(g_wind2[i].times('matplotlib'), data_ev2_bp[i].data, 'purple', label = "bp100-250")
    


In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' , '7F.B13..BHZ', '7F.B19..BHZ', '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' ,'7F.B13..BHZ','7F.B19..BHZ' '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())


fig, ax = plt.subplots(len(events.trace_ids.iloc[event_idx]), figsize=(30, 75), sharex=True, sharey=False)

#fig.text(s='duration (s) = '+str(events.duration), x=0.5, y=0.85, fontsize=15)
#fig.text(s='coincidence sum = '+str(events.coincidence_sum), x=0.5, y=0.69, fontsize=15)
#fig.text(s='starttime = '+str(start_time), x=0.35, y=0.525, fontsize=15)
#props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#for i in np.arange(0, len(g_wind1), 1):
for id in events.trace_ids.iloc[event_idx]:
    i = int(axes_id[id])
    print(i)
    # plot waveform
    ax[i].plot(g_wind1[i].times('matplotlib'), g_wind1[i].data, color='limegreen', label="event_unfiltered")
    ax[i].plot(data_hp350[i].times('matplotlib'), data_hp350[i].data, 'blue', label = "hp350")
    ax[i].plot(g_wind1[i].times('matplotlib'), data_bp[i].data, 'purple', label = "bp100-250")



    # plot geophone name
    ax[i].text(1, 15000, list(axes_id.keys())[i])
    #ax[i].text(x=1.0, y=20000.0, s="LALALA")
    #ax[i].text(0.1, 0.9,'matplotlib', ha='center', va='center', transform=ax.transAxes)
ax[i].legend(loc="upper right")
ax[i].xaxis.set_major_formatter(PrecisionDateFormatter("%H:%M:%S.{ms}"))
    
## TRYING TO PLOT WHEN TRIGGERED   
#for j in range(len(df_time)):
#    for id in events.trace_ids.iloc[j]:
#        i = int(axes_id[id])
#        ax[i].axvspan(df_time[j].timestamp - times_stamp[0][0], df_time[j].timestamp + events.duration[j] - times_stamp[0][0], ymin=0.1, ymax=0.9, alpha=0.25, color='red')


In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' , '7F.B13..BHZ', '7F.B19..BHZ', '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' ,'7F.B13..BHZ','7F.B19..BHZ' '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())


fig, ax = plt.subplots(14, figsize=(30, 75), sharex=True, sharey=False)

#fig.text(s='duration (s) = '+str(events.duration), x=0.5, y=0.85, fontsize=15)
#fig.text(s='coincidence sum = '+str(events.coincidence_sum), x=0.5, y=0.69, fontsize=15)
#fig.text(s='starttime = '+str(start_time), x=0.35, y=0.525, fontsize=15)
#props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#for i in np.arange(0, len(g_wind1), 1):
for id in events.trace_ids.iloc[event_idx1]:
    i = int(axes_id[id])
    print(i)
    # plot waveform
    ax[i].plot(g_wind2[i].times('matplotlib'), g_wind2[i].data, color='limegreen', label="event_unfiltered")
    ax[i].plot(data_ev2_hp350[i].times('matplotlib'), data_ev2_hp350[i].data, 'blue', label = "hp350")
    ax[i].plot(g_wind2[i].times('matplotlib'), data_ev2_bp[i].data, 'purple', label = "bp100-250")



In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
for id in events.trace_ids.iloc[event_idx1]:
    i = int(axes_id[id]) + 1
    print(id)
    print(i)

In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
for id in events.trace_ids.iloc[event_idx1]:
    i = int(axes_id[id]) + 1
    print(id)
    print(i)

In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' , '7F.B13..BHZ', '7F.B19..BHZ', '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' ,'7F.B13..BHZ','7F.B19..BHZ' '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())


fig, ax = plt.subplots(len(events.trace_ids.iloc[event_idx]), figsize=(30, 75), sharex=True, sharey=False)

#fig.text(s='duration (s) = '+str(events.duration), x=0.5, y=0.85, fontsize=15)
#fig.text(s='coincidence sum = '+str(events.coincidence_sum), x=0.5, y=0.69, fontsize=15)
#fig.text(s='starttime = '+str(start_time), x=0.35, y=0.525, fontsize=15)
#props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#for i in np.arange(0, len(g_wind1), 1):
for id in events.trace_ids.iloc[event_idx]:
    i = int(axes_id[id])
    # plot waveform
    ax[i].plot(g_wind3[i].times('matplotlib'), g_wind3[i].data, color='limegreen', label="event_unfiltered")
    ax[i].plot(data_ev3_hp350[i].times('matplotlib'), data_ev3_hp350[i].data, 'blue', label = "hp350")
    ax[i].plot(g_wind3[i].times('matplotlib'), data_ev3_bp[i].data, 'purple', label = "bp100-250")



    # plot geophone name
    ax[i].text(1, 15000, list(axes_id.keys())[i])
    #ax[i].text(x=1.0, y=20000.0, s="LALALA")
    #ax[i].text(0.1, 0.9,'matplotlib', ha='center', va='center', transform=ax.transAxes)
ax[i].legend(loc="upper right")
ax[i].xaxis.set_major_formatter(PrecisionDateFormatter("%H:%M:%S.{ms}"))

In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' , '7F.B13..BHZ', '7F.B19..BHZ', '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' ,'7F.B13..BHZ','7F.B19..BHZ' '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())


fig, ax = plt.subplots(len(events.trace_ids.iloc[event_idx]), figsize=(30, 75), sharex=True, sharey=False)

#fig.text(s='duration (s) = '+str(events.duration), x=0.5, y=0.85, fontsize=15)
#fig.text(s='coincidence sum = '+str(events.coincidence_sum), x=0.5, y=0.69, fontsize=15)
#fig.text(s='starttime = '+str(start_time), x=0.35, y=0.525, fontsize=15)
#props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#for i in np.arange(0, len(g_wind1), 1):
for id in events.trace_ids.iloc[event_idx]:
    i = int(axes_id[id])
    # plot waveform
    ax[i].plot(g_wind4[i].times('matplotlib'), g_wind4[i].data, color='limegreen', label="event_unfiltered")
    ax[i].plot(data_ev4_hp350[i].times('matplotlib'), data_ev4_hp350[i].data, 'blue', label = "hp350")
    ax[i].plot(g_wind4[i].times('matplotlib'), data_ev4_bp[i].data, 'purple', label = "bp100-250")



    # plot geophone name
    ax[i].text(1, 15000, list(axes_id.keys())[i])
    ax[i].text(1, 3000, r'an equation: $E=mc^2$', fontsize=15)
    #ax[i].text(x=1.0, y=20000.0, s="LALALA")
    #ax[i].text(0.1, 0.9,'matplotlib', ha='center', va='center', transform=ax.transAxes)
ax[i].legend(loc="upper right")
ax[i].xaxis.set_major_formatter(PrecisionDateFormatter("%H:%M:%S.{ms}"))

In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())
#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' , '7F.B13..BHZ', '7F.B19..BHZ', '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

#axes_id = dict(np.array([['7F.B01..BHZ' ,'7F.B02..BHZ' , '7F.B03..BHZ' , '7F.B04..BHZ' , '7F.B06..BHZ' , '7F.B07..BHZ' , '7F.B08..BHZ' , '7F.B09..BHZ'  , '7F.B10..BHZ' , '7F.B11..BHZ', '7F.B12..BHZ' ,'7F.B13..BHZ','7F.B19..BHZ' '7F.B20..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())


fig, ax = plt.subplots(len(events.trace_ids.iloc[event_idx]), figsize=(30, 75), sharex=True, sharey=False)

#fig.text(s='duration (s) = '+str(events.duration), x=0.5, y=0.85, fontsize=15)
#fig.text(s='coincidence sum = '+str(events.coincidence_sum), x=0.5, y=0.69, fontsize=15)
#fig.text(s='starttime = '+str(start_time), x=0.35, y=0.525, fontsize=15)
#props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#for i in np.arange(0, len(g_wind1), 1):
for id in events.trace_ids.iloc[event_idx]:
    i = int(axes_id[id])
    # plot waveform
    ax[i].plot(g_wind1[i].times('matplotlib'), g_wind1[i].data, color='limegreen', label="event_unfiltered")
    ax[i].plot(data_hp350[i].times('matplotlib'), data_hp350[i].data, 'blue', label = "hp350")
    ax[i].plot(g_wind1[i].times('matplotlib'), data_bp[i].data, 'purple', label = "bp100-250")



    # plot geophone name
    ax[i].text(1, 15000, list(axes_id.keys())[i])
    #ax[i].text(x=1.0, y=20000.0, s="LALALA")
    #ax[i].text(0.1, 0.9,'matplotlib', ha='center', va='center', transform=ax.transAxes)
ax[i].legend(loc="upper right")
ax[i].xaxis.set_major_formatter(PrecisionDateFormatter("%H:%M:%S.{ms}"))

In [ ]:
len(data_hp350[0])

In [ ]:
plt.figure()
plt.plot(g_wind1[1].times('matplotlib'),g_wind1[0].data, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(g_wind1[1].times('matplotlib'), data_hp350[0].data, 'blue', label = "hp350")
plt.plot(g_wind1[1].times('matplotlib'), data_bp[0].data, 'purple', label = "bp100-250")

plt.legend(loc="upper right")

In [ ]:
g_wind1[i].times('matplotlib')

In [ ]:
plt.figure()
plt.plot(g_wind1[1].times('matplotlib'),g_wind1[1].data, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(g_wind1[1].times('matplotlib'), data_hp350[1].data, 'blue', label = "hp350")
plt.plot(g_wind1[1].times('matplotlib'), data_bp[1].data, 'purple', label = "bp100-250")

plt.legend(loc="upper right")

In [ ]:
plt.figure()
plt.plot(g_wind1[1].times('matplotlib'),g_wind1[2].data, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(g_wind1[1].times('matplotlib'), data_hp350[2].data, 'blue', label = "hp350")
plt.plot(g_wind1[1].times('matplotlib'), data_bp[2].data, 'purple', label = "bp100-250")

plt.legend(loc="upper right")

In [ ]:
plt.figure()
plt.plot(g_wind1[1].times('matplotlib'),g_wind1[3].data, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(g_wind1[1].times('matplotlib'), data_hp350[3].data, 'blue', label = "hp350")
plt.plot(g_wind1[1].times('matplotlib'), data_bp[3].data, 'purple', label = "bp100-250")

plt.legend(loc="upper right")

In [ ]:
plt.figure()
plt.plot(g_wind1[1].times('matplotlib'),g_wind1[4].data, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(g_wind1[1].times('matplotlib'), data_hp350[4].data, 'blue', label = "hp350")
plt.plot(g_wind1[1].times('matplotlib'), data_bp[4].data, 'purple', label = "bp100-250")

plt.legend(loc="upper right")

In [ ]:
plt.figure()
plt.plot(g_wind1[1].times('matplotlib'),g_wind1[5].data, 'limegreen', label = "main event")
#plt.plot(x,data_lp_hp,'green',label = "lp hp")
#plt.plot(x, data_bp, 'blue', label = "bp")
plt.plot(g_wind1[1].times('matplotlib'), data_hp350[5].data, 'blue', label = "hp350")
plt.plot(g_wind1[1].times('matplotlib'), data_bp[5].data, 'purple', label = "bp100-250")

plt.legend(loc="upper right")